# LSTM notebook

## Load Dataset

In [1]:
from datasets import load_dataset
import numpy as np
import pandas as pd

train_dataset = load_dataset("snli", split='train').to_pandas()
test_dataset = load_dataset("snli", split='test').to_pandas()
val_dataset = load_dataset("snli", split='validation').to_pandas()

Found cached dataset snli (/Users/pepegarcia/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)
Found cached dataset snli (/Users/pepegarcia/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)
Found cached dataset snli (/Users/pepegarcia/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)


In [2]:
from data_utils_lstm import download_data
download_data()

glove.6B.zip already exists.
Extracting...
done!


## Preprocessing

In [3]:
from data_utils_lstm import Filter
filter = Filter()
train_dataset_filtered= filter.transform(train_dataset)
test_dataset_filtered = filter.transform(test_dataset)
validation_dataset_filtered = filter.transform(val_dataset)

In [4]:
from data_utils_lstm import load_glove_embeddings

glove_embeddings_path = './data/glove.6B.300d.txt'
glove_embeddings = load_glove_embeddings(glove_embeddings_path)

In [5]:
from data_utils_lstm import preprocess_dataset

train_dataset_preprocessed = preprocess_dataset(train_dataset_filtered, glove_embeddings)
test_dataset_preprocessed = preprocess_dataset(test_dataset_filtered, glove_embeddings)
val_dataset_preprocessed = preprocess_dataset(validation_dataset_filtered, glove_embeddings)

/Users/pepegarcia/Documents/GitHub/NLI/src/alternative_models/LSTM/data_utils_lstm.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["premise_embedding"] = premise_embeddings
/Users/pepegarcia/Documents/GitHub/NLI/src/alternative_models/LSTM/data_utils_lstm.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["hypothesis_embedding"] = hypothesis_embeddings
/Users/pepegarcia/Documents/GitHub/NLI/src/alternative_models/LSTM/data_utils_lstm.py:83: SettingWithCopyWarning: 
A value is trying t

## Model

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Embedding, Input, Reshape, Dropout

# Define input shape
input_shape = (300,)

# Define model architecture with dropout
input_layer = Input(shape=input_shape)
reshape_layer = Reshape((1, input_shape[0]))(input_layer)
lstm_layer = LSTM(64, dropout=0.2)(reshape_layer)
dropout_layer = Dropout(0.2)(lstm_layer)
output_layer = Dense(3, activation='softmax')(dropout_layer)

model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

# Train the model
X_train = np.array(train_dataset_preprocessed['premise_embedding'].tolist())
X_val = np.array(val_dataset_preprocessed['premise_embedding'].tolist())
y_train = pd.get_dummies(train_dataset_preprocessed['label']).values
y_val = pd.get_dummies(val_dataset_preprocessed['label']).values

model.fit(X_train, y_train, validation_data=(
    X_val, y_val), epochs=10, batch_size=32)

model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])


2023-03-28 01:33:34.716160: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-28 01:33:39.508105: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-28 01:33:39.510014: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split

Epoch 1/10


2023-03-28 01:33:41.506677: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-28 01:33:41.508838: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-28 01:33:41.510792: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

17167/17168 [============================>.] - ETA: 0s - loss: 1.0988 - accuracy: 0.3333

2023-03-28 01:34:21.335743: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-28 01:34:21.336927: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-28 01:34:21.338660: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

17168/17168 [==============================] - 41s 2ms/step - loss: 1.0988 - accuracy: 0.3333 - val_loss: 1.0986 - val_accuracy: 0.3382
Epoch 2/10
17168/17168 [==============================] - 37s 2ms/step - loss: 1.0987 - accuracy: 0.3341 - val_loss: 1.0986 - val_accuracy: 0.3382
Epoch 3/10
17168/17168 [==============================] - 38s 2ms/step - loss: 1.0987 - accuracy: 0.3330 - val_loss: 1.0987 - val_accuracy: 0.3383
Epoch 4/10
17168/17168 [==============================] - 88s 5ms/step - loss: 1.0987 - accuracy: 0.3323 - val_loss: 1.0985 - val_accuracy: 0.3382
Epoch 5/10
17168/17168 [==============================] - 126s 7ms/step - loss: 1.0987 - accuracy: 0.3329 - val_loss: 1.0988 - val_accuracy: 0.3285
Epoch 6/10
17168/17168 [==============================] - 38s 2ms/step - loss: 1.0987 - accuracy: 0.3338 - val_loss: 1.0986 - val_accuracy: 0.3382
Epoch 7/10
17168/17168 [==============================] - 39s 2ms/step - loss: 1.0987 - accuracy: 0.3335 - val_loss: 1.0987 - va